In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datatable as dt
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from datatable.models import Ftrl
import pickle
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import xgboost as xgb
from sklearn.metrics import make_scorer, matthews_corrcoef
from datetime import datetime
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
import lightgbm as lgb
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
df = pd.read_csv('final_df_train_3.0.csv',nrows=300000)
df.fillna(0,inplace=True)

In [3]:
df.drop(['Unnamed: 0','kurtosis','max_num_value','skewness','skewness_imp_st','mean_num_value','sum_num_value_imp_st','sum_num_value','mean_num_value_imp_st'],axis=1,inplace=True)

In [4]:
df.head()

,Id,no_of_category,different_category,probability_value,L0_S0_F0,L0_S0_F2,L0_S0_F4,L0_S0_F6,L0_S0_F10,L0_S0_F14,...,fraction_of_measurementsL3,min_num_value,max_num_value_imp_st,min_num_value_imp_st,kurtosis_imp_st,error_rate_on_imp_stations,mintime_imp_st,maxtime_imp_st,time_difference_imp_st,total_no_of_different_date
0,4,0,0,-0.000818,0.030,-0.034,-0.197,-0.179,0.116,-0.032,...,0.483516,-0.702,0.520,-0.702,46.537694,0.3,87.26,87.29,0.03,8
1,6,0,0,-0.000818,0.000,0.000,0.000,0.000,0.000,0.000,...,0.586081,-0.480,0.255,-0.480,28.652446,0.3,1315.73,1315.75,0.02,6
2,7,63,1,0.000415,0.088,0.086,0.003,-0.052,0.025,-0.072,...,0.586081,-0.480,0.238,-0.480,36.028735,0.3,1624.40,1624.42,0.02,7
3,9,63,1,0.000415,-0.036,-0.064,0.294,0.330,0.161,0.128,...,0.586081,-0.480,0.218,-0.480,37.098558,0.4,1154.12,1154.16,0.04,9
4,11,63,1,0.000415,-0.055,-0.086,0.294,0.330,0.025,0.168,...,0.586081,-0.347,0.520,-0.347,32.536342,0.4,606.00,606.02,0.02,7


In [5]:
X = df.drop(["Id","Response"], axis=1)
y = df['Response']

In [6]:
#credit:-https://www.youtube.com/watch?v=9HomdnM12o4&t=716s
def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))

In [9]:
param_grid = {'gamma': [0,0.1,0.2,0.3,0.4,0.8,1.6,3.2,6.4,12.8,25.6,51.2,102.4, 200],
              'learning_rate': [0.01, 0.03, 0.05, 0.1, 0.15, 0.2, 0.25, 0.300000012, 0.4, 0.5, 0.6, 0.7],
              'max_depth': [3,5,6,7,8,9,10,11,12,15,17,20,23,25],
              'n_estimators': [50,65,80,100,115,130,150],
              'reg_alpha': [0,0.1,0.2,0.4,0.8,1.6,3.2,6.4,12.8,25.6,51.2,102.4,200],
              'reg_lambda': [0,0.1,0.2,0.4,0.8,1.6,3.2,6.4,12.8,25.6,51.2,102.4,200],
             "min_child_weight" : [ 1, 3, 5, 7, 9,12,15],
             "colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7]}

In [11]:
mcc = make_scorer(matthews_corrcoef)
classifier=xgb.XGBClassifier()
random_search=RandomizedSearchCV(classifier,param_distributions=param_grid,n_iter=5,scoring=mcc,cv=5,verbose=3)

In [12]:
import warnings
warnings.filterwarnings('ignore')
# Here we go
start_time = timer(None) # timing starts from this point for "start_time" variable
random_search.fit(X,y,eval_metric = 'auc')
timer(start_time) # timing ends here for "start_time" variable

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV 1/5] END colsample_bytree=0.3, gamma=1.6, learning_rate=0.4, max_depth=7, min_child_weight=1, n_estimators=150, reg_alpha=25.6, reg_lambda=25.6;, score=0.262 total time=  15.4s
[CV 2/5] END colsample_bytree=0.3, gamma=1.6, learning_rate=0.4, max_depth=7, min_child_weight=1, n_estimators=150, reg_alpha=25.6, reg_lambda=25.6;, score=0.315 total time=  15.5s
[CV 3/5] END colsample_bytree=0.3, gamma=1.6, learning_rate=0.4, max_depth=7, min_child_weight=1, n_estimators=150, reg_alpha=25.6, reg_lambda=25.6;, score=0.297 total time=  16.1s
[CV 4/5] END colsample_bytree=0.3, gamma=1.6, learning_rate=0.4, max_depth=7, min_child_weight=1, n_estimators=150, reg_alpha=25.6, reg_lambda=25.6;, score=0.299 total time=  16.1s
[CV 5/5] END colsample_bytree=0.3, gamma=1.6, learning_rate=0.4, max_depth=7, min_child_weight=1, n_estimators=150, reg_alpha=25.6, reg_lambda=25.6;, score=0.323 total time=  15.1s
[CV 1/5] END colsample_bytree=0.7, 

In [13]:
random_search.best_params_

{'reg_lambda': 0.2,
 'reg_alpha': 1.6,
 'n_estimators': 65,
 'min_child_weight': 12,
 'max_depth': 3,
 'learning_rate': 0.5,
 'gamma': 3.2,
 'colsample_bytree': 0.3}

# Tuning Random Forest

In [7]:
params_rf = {'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}

In [8]:
rf=RandomForestClassifier()
random_search_rf=RandomizedSearchCV(rf,param_distributions=params_rf,n_iter=5,scoring='roc_auc',cv=5,verbose=3)

In [9]:
start_time = timer(None) 
random_search_rf.fit(X,y)
timer(start_time)

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV 1/5] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=1800;, score=0.814 total time=  31.0s
[CV 2/5] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=1800;, score=0.952 total time=  31.4s
[CV 3/5] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=1800;, score=0.802 total time=  30.8s
[CV 4/5] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=1800;, score=0.810 total time=  30.9s
[CV 5/5] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=1800;, score=0.692 total time=  31.0s
[CV 1/5] END bootstrap=False, max_depth=40, max_features=auto, min_samples_leaf=4, min_samples_split=10, n_estimators=2000;, score=0.839 total time= 1.3min
[CV 2/5] 

In [10]:
random_search_rf.best_params_

{'n_estimators': 1800,
 'min_samples_split': 10,
 'min_samples_leaf': 2,
 'max_features': 'sqrt',
 'max_depth': 10,
 'bootstrap': True}

# Tuning Light GBM

In [14]:
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform
param_lgm ={'num_leaves': sp_randint(6, 50), 
             'min_child_samples': sp_randint(100, 500), 
             'min_child_weight': [1e-5, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4],
             'subsample': sp_uniform(loc=0.2, scale=0.8), 
             'colsample_bytree': sp_uniform(loc=0.4, scale=0.6),
             'reg_alpha': [0, 1e-1, 1, 2, 5, 7, 10, 50, 100],
             'reg_lambda': [0, 1e-1, 1, 5, 10, 20, 50, 100]}

In [15]:
#mcc = make_scorer(matthews_corrcoef)
clf = lgb.LGBMClassifier()
random_search_lgm = RandomizedSearchCV(
    estimator=clf, param_distributions=param_lgm, 
    n_iter=5,
    scoring='roc_auc',
    cv=5,
    verbose=3)

In [17]:
start_time = timer(None) # timing starts from this point for "start_time" variable
random_search_lgm.fit(X,y)
timer(start_time)

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV 1/5] END colsample_bytree=0.43455527965836527, min_child_samples=198, min_child_weight=10.0, num_leaves=12, reg_alpha=0.1, reg_lambda=0.1, subsample=0.36181919064442924;, score=0.787 total time=   4.9s
[CV 2/5] END colsample_bytree=0.43455527965836527, min_child_samples=198, min_child_weight=10.0, num_leaves=12, reg_alpha=0.1, reg_lambda=0.1, subsample=0.36181919064442924;, score=0.808 total time=   5.2s
[CV 3/5] END colsample_bytree=0.43455527965836527, min_child_samples=198, min_child_weight=10.0, num_leaves=12, reg_alpha=0.1, reg_lambda=0.1, subsample=0.36181919064442924;, score=0.820 total time=   4.8s
[CV 4/5] END colsample_bytree=0.43455527965836527, min_child_samples=198, min_child_weight=10.0, num_leaves=12, reg_alpha=0.1, reg_lambda=0.1, subsample=0.36181919064442924;, score=0.802 total time=   4.7s
[CV 5/5] END colsample_bytree=0.43455527965836527, min_child_samples=198, min_child_weight=10.0, num_leaves=12, reg_

In [18]:
random_search_lgm.best_params_

{'colsample_bytree': 0.43455527965836527,
 'min_child_samples': 198,
 'min_child_weight': 10.0,
 'num_leaves': 12,
 'reg_alpha': 0.1,
 'reg_lambda': 0.1,
 'subsample': 0.36181919064442924}

# Tuning DecisionTreeClassifier

In [19]:
param_dt={"splitter":["best","random"],
            "max_depth" : [1,3,5,7,9,11,12],
           "min_samples_leaf":[1,2,3,4,5,6,7,8,9,10],
           "min_weight_fraction_leaf":[0.1,0.2,0.3,0.4,0.5],
           "max_features":["auto","log2","sqrt",None],
           "max_leaf_nodes":[None,10,20,30,40,50,60,70,80,90] }

In [20]:
#mcc = make_scorer(matthews_corrcoef)
dt=DecisionTreeClassifier()
random_search_dt=RandomizedSearchCV(dt,param_distributions=param_dt,n_iter=10,scoring='roc_auc',cv=10,verbose=3)

In [22]:
start_time = timer(None) # timing starts from this point for "start_time" variable
random_search_dt.fit(X,y)
timer(start_time)

Fitting 10 folds for each of 10 candidates, totalling 100 fits
[CV 1/10] END max_depth=5, max_features=log2, max_leaf_nodes=60, min_samples_leaf=8, min_weight_fraction_leaf=0.3, splitter=random;, score=0.500 total time=   0.6s
[CV 2/10] END max_depth=5, max_features=log2, max_leaf_nodes=60, min_samples_leaf=8, min_weight_fraction_leaf=0.3, splitter=random;, score=0.500 total time=   0.7s
[CV 3/10] END max_depth=5, max_features=log2, max_leaf_nodes=60, min_samples_leaf=8, min_weight_fraction_leaf=0.3, splitter=random;, score=0.552 total time=   0.7s
[CV 4/10] END max_depth=5, max_features=log2, max_leaf_nodes=60, min_samples_leaf=8, min_weight_fraction_leaf=0.3, splitter=random;, score=0.500 total time=   0.7s
[CV 5/10] END max_depth=5, max_features=log2, max_leaf_nodes=60, min_samples_leaf=8, min_weight_fraction_leaf=0.3, splitter=random;, score=0.500 total time=   0.7s
[CV 6/10] END max_depth=5, max_features=log2, max_leaf_nodes=60, min_samples_leaf=8, min_weight_fraction_leaf=0.3, sp

[CV 1/10] END max_depth=3, max_features=log2, max_leaf_nodes=40, min_samples_leaf=6, min_weight_fraction_leaf=0.3, splitter=random;, score=0.515 total time=   0.7s
[CV 2/10] END max_depth=3, max_features=log2, max_leaf_nodes=40, min_samples_leaf=6, min_weight_fraction_leaf=0.3, splitter=random;, score=0.500 total time=   0.9s
[CV 3/10] END max_depth=3, max_features=log2, max_leaf_nodes=40, min_samples_leaf=6, min_weight_fraction_leaf=0.3, splitter=random;, score=0.500 total time=   0.8s
[CV 4/10] END max_depth=3, max_features=log2, max_leaf_nodes=40, min_samples_leaf=6, min_weight_fraction_leaf=0.3, splitter=random;, score=0.573 total time=   0.8s
[CV 5/10] END max_depth=3, max_features=log2, max_leaf_nodes=40, min_samples_leaf=6, min_weight_fraction_leaf=0.3, splitter=random;, score=0.500 total time=   0.8s
[CV 6/10] END max_depth=3, max_features=log2, max_leaf_nodes=40, min_samples_leaf=6, min_weight_fraction_leaf=0.3, splitter=random;, score=0.500 total time=   0.8s
[CV 7/10] END ma


 Time taken: 0 hours 2 minutes and 4.32 seconds.


In [23]:
random_search_dt.best_params_

{'splitter': 'best',
 'min_weight_fraction_leaf': 0.1,
 'min_samples_leaf': 1,
 'max_leaf_nodes': 30,
 'max_features': 'sqrt',
 'max_depth': 9}

# Tuning Adaboost 

In [29]:
param_adb = {
    'n_estimators': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 20],
    'learning_rate': [(0.97 + x / 100) for x in range(0, 8)],
    'algorithm': ['SAMME', 'SAMME.R']
}

In [30]:
adb = AdaBoostClassifier(random_state=42)
random_search_adb=RandomizedSearchCV(adb,param_distributions=param_adb,n_iter=5,scoring='roc_auc',cv=5,verbose=3)

In [31]:
start_time = timer(None) # timing starts from this point for "start_time" variable
random_search_adb.fit(X,y)
timer(start_time)

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV 1/5] END algorithm=SAMME.R, learning_rate=1.01, n_estimators=5;, score=0.774 total time=  15.6s
[CV 2/5] END algorithm=SAMME.R, learning_rate=1.01, n_estimators=5;, score=0.804 total time=  11.9s
[CV 3/5] END algorithm=SAMME.R, learning_rate=1.01, n_estimators=5;, score=0.808 total time=  11.7s
[CV 4/5] END algorithm=SAMME.R, learning_rate=1.01, n_estimators=5;, score=0.785 total time=  11.6s
[CV 5/5] END algorithm=SAMME.R, learning_rate=1.01, n_estimators=5;, score=0.798 total time=  11.6s
[CV 1/5] END algorithm=SAMME.R, learning_rate=0.97, n_estimators=7;, score=0.777 total time=  16.1s
[CV 2/5] END algorithm=SAMME.R, learning_rate=0.97, n_estimators=7;, score=0.804 total time=  16.1s
[CV 3/5] END algorithm=SAMME.R, learning_rate=0.97, n_estimators=7;, score=0.811 total time=  16.1s
[CV 4/5] END algorithm=SAMME.R, learning_rate=0.97, n_estimators=7;, score=0.787 total time=  16.2s
[CV 5/5] END algorithm=SAMME.R, learning

In [32]:
random_search_adb.best_params_

{'n_estimators': 7, 'learning_rate': 0.97, 'algorithm': 'SAMME.R'}

# Tuning SVM

In [7]:
param_svm = {'C': [0.1, 1, 10, 100, 1000],
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf']}

In [8]:
#mcc = make_scorer(matthews_corrcoef)
from sklearn.svm import SVC
s=SVC()
random_search_svc=RandomizedSearchCV(s,param_distributions=param_svm,n_iter=5,scoring='roc_auc',cv=5,verbose=3)

In [9]:
start_time = timer(None) # timing starts from this point for "start_time" variable
random_search_svc.fit(X,y)
timer(start_time)

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV 1/5] END .........C=10, gamma=1, kernel=rbf;, score=0.500 total time=  43.0s
[CV 2/5] END .........C=10, gamma=1, kernel=rbf;, score=0.500 total time=  40.0s
[CV 3/5] END .........C=10, gamma=1, kernel=rbf;, score=0.500 total time=  38.1s
[CV 4/5] END .........C=10, gamma=1, kernel=rbf;, score=0.500 total time=  35.5s
[CV 5/5] END .........C=10, gamma=1, kernel=rbf;, score=0.500 total time=  39.9s
[CV 1/5] END ....C=10, gamma=0.0001, kernel=rbf;, score=0.502 total time=  48.5s
[CV 2/5] END ....C=10, gamma=0.0001, kernel=rbf;, score=0.503 total time=  39.8s
[CV 3/5] END ....C=10, gamma=0.0001, kernel=rbf;, score=0.503 total time=  34.4s
[CV 4/5] END ....C=10, gamma=0.0001, kernel=rbf;, score=0.504 total time=  36.2s
[CV 5/5] END ....C=10, gamma=0.0001, kernel=rbf;, score=0.503 total time=  33.7s
[CV 1/5] END .....C=1000, gamma=0.1, kernel=rbf;, score=0.500 total time=  36.0s
[CV 2/5] END .....C=1000, gamma=0.1, kernel=rbf;,

In [10]:
random_search_svc.best_params_

{'kernel': 'rbf', 'gamma': 0.0001, 'C': 10}

# Tuning GBDT

In [7]:
param_gbdt = {'learning_rate': [1, 0.5, 0.25, 0.1, 0.05, 0.01],
              'n_estimators': [1, 2, 4, 8, 16, 32, 64, 100, 200],
              'max_depth': np.linspace(1, 32, 32, endpoint=True),
             'min_samples_split' : np.linspace(0.1, 1.0, 10, endpoint=True),
             'min_samples_leaf' : np.linspace(0.1, 0.5, 5, endpoint=True),
             'max_features' :  list(range(1,10,2))
            }

In [8]:
from sklearn.ensemble import GradientBoostingClassifier
gbdt = GradientBoostingClassifier()
random_search_gbdt=RandomizedSearchCV(gbdt,param_distributions=param_gbdt,n_iter=5,scoring='roc_auc',cv=5,verbose=3)

In [9]:
start_time = timer(None) # timing starts from this point for "start_time" variable
random_search_gbdt.fit(X,y)
timer(start_time)

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV 1/5] END learning_rate=1, max_depth=24.0, max_features=7, min_samples_leaf=0.1, min_samples_split=0.1, n_estimators=2;, score=0.615 total time=   1.1s
[CV 2/5] END learning_rate=1, max_depth=24.0, max_features=7, min_samples_leaf=0.1, min_samples_split=0.1, n_estimators=2;, score=0.655 total time=   1.3s
[CV 3/5] END learning_rate=1, max_depth=24.0, max_features=7, min_samples_leaf=0.1, min_samples_split=0.1, n_estimators=2;, score=0.665 total time=   1.2s
[CV 4/5] END learning_rate=1, max_depth=24.0, max_features=7, min_samples_leaf=0.1, min_samples_split=0.1, n_estimators=2;, score=0.651 total time=   1.1s
[CV 5/5] END learning_rate=1, max_depth=24.0, max_features=7, min_samples_leaf=0.1, min_samples_split=0.1, n_estimators=2;, score=0.637 total time=   1.1s
[CV 1/5] END learning_rate=1, max_depth=21.0, max_features=3, min_samples_leaf=0.1, min_samples_split=0.8, n_estimators=1;, score=0.530 total time=   0.8s
[CV 2/5] E

In [10]:
random_search_gbdt.best_params_

{'n_estimators': 16,
 'min_samples_split': 0.5,
 'min_samples_leaf': 0.2,
 'max_features': 7,
 'max_depth': 21.0,
 'learning_rate': 1}

# Predicting with Best Parameters

In [2]:
df = pd.read_csv('final_df_train_3.0.csv')
df.fillna(0,inplace=True)

In [3]:
df.drop(['mindate_id_diff_reverse','mindate_id_diff','Unnamed: 0','kurtosis','max_num_value','skewness','skewness_imp_st','mean_num_value','sum_num_value_imp_st','sum_num_value','mean_num_value_imp_st'],axis=1,inplace=True)

In [4]:
df_test = pd.read_csv('final_df_test_3.0.csv')
df_test.fillna(0,inplace=True)
df_test.drop(['mindate_id_diff_reverse','mindate_id_diff','Unnamed: 0','kurtosis','max_num_value','skewness','skewness_imp_st','mean_num_value','sum_num_value_imp_st','sum_num_value','mean_num_value_imp_st'],axis=1,inplace=True)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(["Id","Response"], axis=1),df["Response"],test_size=0.2, random_state=42, shuffle=True)

In [6]:
model_xgb=xgb.XGBClassifier(booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.3, gamma=3.2, learning_rate=0.5,
       max_delta_step=0, max_depth=3, min_child_weight=12, missing=1,
       n_estimators=65, n_jobs=1,random_state=0,
       objective='binary:logistic', reg_alpha=1.6,
       reg_lambda=0.2, scale_pos_weight=1, seed=None, silent=True,
       subsample=1,use_label_encoder=False)

In [7]:
model_xgb.fit(X_train,y_train)

[17:44:32] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[17:44:43] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.3, gamma=3.2, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.5, max_delta_step=0, max_depth=3,
              min_child_weight=12, missing=1, monotone_constraints='()',
              n_estimators=65, n_jobs=1, num_parallel_tree=1, random_state=0,
              reg_alpha=1.6, reg_lambda=0.2, scale_pos_weight=1, seed=0,
              silent=True, subsample=1, tree_method='exact',
              use_label_encoder=False, validate_parameters=1, verbosity=None)

In [8]:
y_pred_train = model_xgb.predict(X_train)
matthews_corrcoef(y_train,y_pred_train)

0.22742238704879816

In [9]:
y_pred_test = model_xgb.predict(X_test)
matthews_corrcoef(y_test,y_pred_test)

0.1976596290376057

In [8]:
y_pred_train = model_xgb.predict(X_train)
matthews_corrcoef(y_train,y_pred_train)

0.32553513989186117

In [9]:
y_pred_test = model_xgb.predict(X_test)
matthews_corrcoef(y_test,y_pred_test)

0.3034725677187602

In [11]:
y_pred = model_xgb.predict(df_test.drop("Id",axis=1))

In [13]:
#kaggle score 0.30703
c=0
for i in y_pred:
    if i == 1:
        c+=1
c

953

In [14]:
submission = pd.DataFrame({"Id": df_test.Id.values,
                               "Response": y_pred})
submission[['Id', 'Response']].to_csv('xgbsubmission.csv',
                                          index=False)

# Random Forest

In [ ]:
{'n_estimators': 1800,
 'min_samples_split': 10,
 'min_samples_leaf': 2,
 'max_features': 'sqrt',
 'max_depth': 10,
 'bootstrap': True}

In [7]:
model_rf=RandomForestClassifier(n_estimators=1800,min_samples_split=10,min_samples_leaf=2,max_features='sqrt',max_depth=10,bootstrap=True)

In [8]:
model_rf.fit(X_train,y_train)

RandomForestClassifier(max_depth=10, max_features='sqrt', min_samples_leaf=2,
                       min_samples_split=10, n_estimators=1800)

In [9]:
y_pred_train = model_rf.predict(X_train)
matthews_corrcoef(y_train,y_pred_train)

0.23748450253523393

In [10]:
y_pred_test = model_rf.predict(X_test)
matthews_corrcoef(y_test,y_pred_test)

0.19808074357467986

In [11]:
y_pred = model_rf.predict(df_test.drop("Id",axis=1))

In [12]:
c=0
for i in y_pred:
    if i == 1:
        c+=1
c

310

In [13]:
submission = pd.DataFrame({"Id": df_test.Id.values,
                               "Response": y_pred})
submission[['Id', 'Response']].to_csv('rfsubmission.csv',
                                          index=False)

# Light GBM

In [17]:
model_lgb = lgb.LGBMClassifier(colsample_bytree= 0.43455527965836527,
 min_child_samples= 198,
 min_child_weight= 10.0,
 num_leaves= 12,
 reg_alpha= 0.1,
 reg_lambda= 0.1,
 subsample= 0.36181919064442924)

In [18]:
model_lgb.fit(X_train,y_train)

LGBMClassifier(colsample_bytree=0.43455527965836527, min_child_samples=198,
               min_child_weight=10.0, num_leaves=12, reg_alpha=0.1,
               reg_lambda=0.1, subsample=0.36181919064442924)

In [19]:
y_pred_train = model_lgb.predict(X_train)
matthews_corrcoef(y_train,y_pred_train)

0.3211997158771233

In [20]:
y_pred_test = model_lgb.predict(X_test)
matthews_corrcoef(y_test,y_pred_test)

0.3033726979155154

In [21]:
y_pred = model_lgb.predict(df_test.drop("Id",axis=1))

In [22]:
#kaggle score  0.30381
c=0
for i in y_pred:
    if i == 1:
        c+=1
c

941

In [23]:
submission = pd.DataFrame({"Id": df_test.Id.values,
                               "Response": y_pred})
submission[['Id', 'Response']].to_csv('lgbsubmission final'+'.csv',
                                          index=False)

# AdaBoost

In [24]:
adb = AdaBoostClassifier(n_estimators=7,learning_rate=0.97,algorithm='SAMME.R')

In [27]:
adb.fit(X_train,y_train)

AdaBoostClassifier(learning_rate=0.97, n_estimators=7)

In [29]:
y_pred_train = adb.predict(X_train)
matthews_corrcoef(y_train,y_pred_train)

0.22178350926090884

In [30]:
y_pred_test = adb.predict(X_test)
matthews_corrcoef(y_test,y_pred_test)

0.20683896613710062

In [31]:
y_pred = adb.predict(df_test.drop("Id",axis=1))

In [32]:
#kaggle score  0.21444
c=0
for i in y_pred:
    if i == 1:
        c+=1
c

556

In [33]:
submission = pd.DataFrame({"Id": df_test.Id.values,
                               "Response": y_pred})
submission[['Id', 'Response']].to_csv('adbsubmission final'+'.csv',
                                          index=False)

# DecisionTree

In [34]:
model_dt=DecisionTreeClassifier(splitter='best',min_weight_fraction_leaf=0.1,min_samples_leaf=1,max_leaf_nodes=30,max_features='sqrt',max_depth=9)

In [35]:
model_dt.fit(X_train,y_train)

DecisionTreeClassifier(max_depth=9, max_features='sqrt', max_leaf_nodes=30,
                       min_weight_fraction_leaf=0.1)

In [36]:
y_pred_train = model_dt.predict(X_train)
matthews_corrcoef(y_train,y_pred_train)

C:\Users\Debasish Pradhan\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


0.0

In [37]:
y_pred_test = model_dt.predict(X_test)
matthews_corrcoef(y_test,y_pred_test)

C:\Users\Debasish Pradhan\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


0.0

# GBDT

In [6]:
gbdt = GradientBoostingClassifier(n_estimators=16,min_samples_split=0.5,min_samples_leaf=0.2,max_features=7,max_depth=21.0,learning_rate=1)

In [7]:
gbdt.fit(X_train,y_train)

GradientBoostingClassifier(learning_rate=1, max_depth=21.0, max_features=7,
                           min_samples_leaf=0.2, min_samples_split=0.5,
                           n_estimators=16)

In [8]:
y_pred_train = gbdt.predict(X_train)
matthews_corrcoef(y_train,y_pred_train)

C:\Users\Debasish Pradhan\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


0.0

In [9]:
y_pred_test = gbdt.predict(X_test)
matthews_corrcoef(y_test,y_pred_test)

C:\Users\Debasish Pradhan\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


0.0